In [3]:
import graphviz
import heapq
import math
from collections import defaultdict
from collections import deque

In [8]:
class Vertex:
    """A class representing a vertex in a graph."""

    def __init__(self, key):
        """
        Initialize a new Vertex object.

        Args:
            key: The ID/key of the vertex.
        """
        self.id = key
        self.connectedTo = {}

    def addNeighbor(self, nbr, weight=0):
        """
        Add a neighbor to the vertex.

        Args:
            nbr: The ID of the neighbor vertex.
            weight: The weight of the edge between the vertex and its neighbor.
        """
        self.connectedTo[nbr] = weight

    def __str__(self):
        """
        Return a string representation of the vertex.

        Returns:
            A string representation of the vertex.
        """
        return str(self.id)

    def getConnections(self):
        """
        Get the IDs of all the neighbors of the vertex.

        Returns:
            A list of the IDs of all the neighbors of the vertex.
        """
        return self.connectedTo.keys()

    def getId(self):
        """
        Get the ID of the vertex.

        Returns:
            The ID of the vertex.
        """
        return self.id

    def getWeight(self, nbr):
        """
        Get the weight of the edge between the vertex and its neighbor.

        Args:
            nbr: The ID of the neighbor vertex.

        Returns:
            The weight of the edge between the vertex and its neighbor.
        """
        return self.connectedTo[nbr]


class Graph:
    """A class representing a graph."""

    def __init__(self):
        """Initialize a new Graph object."""
        self.vertices = {}

    def addVertex(self, vert):
        """
        Add a vertex to the graph.

        Args:
            vert: The ID of the vertex to add.
        """
        if vert not in self.vertices:
            newVertex = Vertex(vert)
            self.vertices[vert] = newVertex

    def addVerticesFromList(self, vertList):
        """
        Add multiple vertices to the graph from a list.

        Args:
            vertList: A list of vertex IDs to add to the graph.
        """
        for vert in vertList:
            self.addVertex(vert)

    def addEdge(self, fromVert, toVert, weight=1):
        """
        Add an edge between two vertices in the graph.

        Args:
            fromVert: The ID of the vertex the edge starts from.
            toVert: The ID of the vertex the edge ends at.
            weight: The weight of the edge (default 1).
        """
        if fromVert not in self.vertices:
            self.addVertex(fromVert)
        if toVert not in self.vertices:
            self.addVertex(toVert)
        self.vertices[fromVert].addNeighbor(toVert, weight)
        self.vertices[toVert].addNeighbor(fromVert, weight)

    def addEdgesFromList(self, edgeList):
        """
        Add multiple edges to the graph from a list.

        Args:
            edgeList: A list of tuples representing edges in the format (fromVert, toVert) or (fromVert, toVert, weight).
        """
        for edge in edgeList:
            if len(edge) == 2:
                self.addEdge(edge[0], edge[1])
            elif len(edge) == 3:
                self.addEdge(edge[0], edge[1], edge[2])

    def getVertices(self):
        """
        Get a list of all the vertices in the graph.

        Returns:
            A list of all the vertices in the graph.
        """
        return list(self.vertices.keys())

    def getEdges(self):
        """
        Returns a list of all edges in the graph as tuples of the form (vertex1, vertex2).
        """
        edges = []
        for v in self.vertices.values():
            for nbr in v.getConnections():
                edges.append((v.getId(), nbr))
        return edges

    def getNeighbors(self, vertKey):
        """
        Returns a list of all vertices adjacent to the given vertex key.

        Parameters:
        vertKey (str): The key of the vertex to find neighbors for.

        Returns:
        list: A list of vertex keys adjacent to the given vertex.
              Returns None if the vertex is not in the graph.
        """
        if vertKey in self.vertices:
            return list(self.vertices[vertKey].getConnections())
        else:
            return None

    def __contains__(self, vert):
        """
        Returns True if the given vertex is in the graph.

        Parameters:
        vert (Vertex): The vertex to check for in the graph.

        Returns:
        bool: True if the vertex is in the graph, False otherwise.
        """
        return vert in self.vertices

    def saveGraph(self, filename):
        """
        Saves a visualization of the graph to a file with the given name.

        Parameters:
        filename (str): The name of the file to save the graph visualization to.
        """
        dot = graphviz.Graph(comment='Graph')
        for v in self.vertices:
            dot.node(str(v))
        for v in self.vertices.values():
            for nbr in v.getConnections():
                dot.edge(str(v.getId()), str(nbr))
        dot.render(filename, format='png')

    def getShortestPaths(self, fromVert):
        """
        Finds the shortest paths from a given vertex to all other vertices in the graph using Dijkstra's algorithm.

        Parameters:
        fromVert (str): The key of the vertex to find shortest paths from.

        Returns:
        dict: A dictionary of shortest paths, where each key is a vertex key and each value is a tuple of the form
              (distance, path), where distance is the distance from the source vertex to the key vertex and path is a
              list of vertex keys representing the shortest path from the source vertex to the key vertex.
        """
        # Initialize distances and previous nodes
        distances = {v: float('inf') for v in self.vertices}
        distances[fromVert] = 0
        previous = {v: None for v in self.vertices}

        # Initialize heap with source vertex
        heap = [(0, fromVert)]

        # Main loop
        while heap:
            (dist, curr) = heapq.heappop(heap)

            # Check if we already found a better path to curr
            if dist > distances[curr]:
                continue

            # Relax all neighbors of curr
            for neighbor in self.vertices[curr].getConnections():
                newDist = distances[curr] + self.vertices[curr].getWeight(neighbor)
                if newDist < distances[neighbor]:
                    distances[neighbor] = newDist
                    previous[neighbor] = curr
                    heapq.heappush(heap, (newDist, neighbor))

        # Create dictionary of shortest paths
        shortestPaths = {}
        for v in self.vertices:
            path = []
            curr = v
            while curr != fromVert:
                path.insert(0, curr)
                curr = previous[curr]
            path.insert(0, fromVert)
            shortestPaths[v] = (distances[v], path)

        return shortestPaths



In [4]:
g = Graph()
g.addEdge("Alice", "Bob")
g.addEdge("Carl", "Alice")
g.addEdge("Alice", "David")
g.addEdge("Alice", "Ernst")
g.addEdge("Alice", "Frank")
g.addEdge("Bob", "Gail")
g.addEdge("Gail", "Harry")
g.addEdge("Harry", "Jen")
g.addEdge("Jen", "Gail")
g.addEdge("Harry", "Irene")
g.addEdge("Irene", "Gail")
g.addEdge("Irene", "Jen")
g.addEdge("Ernst", "Frank")
g.addEdge("David", "Carl")
g.addEdge("Carl", "Frank")

#g.getShortestPaths('Alice')
shortestPaths = g.getShortestPaths("Alice")

for node in sorted(shortestPaths):
    if shortestPaths[node][0] is None:
        print(f"Shortest path from Alice to {node}:")
        print("Path: Unreachable")
    else:
        print(f"Shortest path from Alice to {node}:")
        print(f"Distance: {shortestPaths[node][0]}")
        print(f"Path: {' -> '.join(shortestPaths[node][1])}")

Shortest path from Alice to Alice:
Distance: 0
Path: Alice
Shortest path from Alice to Bob:
Distance: 1
Path: Alice -> Bob
Shortest path from Alice to Carl:
Distance: 1
Path: Alice -> Carl
Shortest path from Alice to David:
Distance: 1
Path: Alice -> David
Shortest path from Alice to Ernst:
Distance: 1
Path: Alice -> Ernst
Shortest path from Alice to Frank:
Distance: 1
Path: Alice -> Frank
Shortest path from Alice to Gail:
Distance: 2
Path: Alice -> Bob -> Gail
Shortest path from Alice to Harry:
Distance: 3
Path: Alice -> Bob -> Gail -> Harry
Shortest path from Alice to Irene:
Distance: 3
Path: Alice -> Bob -> Gail -> Irene
Shortest path from Alice to Jen:
Distance: 3
Path: Alice -> Bob -> Gail -> Jen


In [5]:
# create a new empty graph
graph = Graph()

# add edges to the graph
graph.addEdgesFromList([
    ("Alice", "Bob"),
    ("Carl", "Alice"),
    ("Alice", "David"),
    ("Alice", "Ernst"),
    ("Alice", "Frank"),
    ("Bob", "Gail"),
    ("Gail", "Harry"),
    ("Harry", "Jen"),
    ("Jen", "Gail"),
    ("Harry", "Irene"),
    ("Irene", "Gail"),
    ("Irene", "Jen"),
    ("Ernst", "Frank"),
    ("David", "Carl"),
    ("Carl", "Frank")
])

# save and render the graph
graph.saveGraph("my_graph")




In [6]:
# create a new empty graph
graph = Graph()

# add vertices to the graph
graph.addVertex("A")
graph.addVertex("B")
graph.addVertex("C")
graph.addVertex("D")
graph.addVertex("E")

# add edges to the graph
graph.addEdge("A", "B")
graph.addEdge("A", "C")
graph.addEdge("B", "D")
graph.addEdge("C", "D")
graph.addEdge("C", "E")

# get all vertices in the graph
print("Vertices in the graph: ", graph.getVertices())

# get all edges in the graph
print("Edges in the graph: ", graph.getEdges())

# get all neighbors of a vertex
print("Neighbors of vertex C: ", graph.getNeighbors("C"))


# check if a vertex is in the graph
print("Is vertex A in the graph? ", "A" in graph)

# save the graph to a file
graph.saveGraph("my_graph.dot")

Vertices in the graph:  ['A', 'B', 'C', 'D', 'E']
Edges in the graph:  [('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'D'), ('C', 'A'), ('C', 'D'), ('C', 'E'), ('D', 'B'), ('D', 'C'), ('E', 'C')]
Neighbors of vertex C:  ['A', 'D', 'E']
Is vertex A in the graph?  True
